### In this notebook I follow the approach described by Avellaneda and Lee (2010) 

# Setup

In [111]:
from binance.client import Client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

In [112]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller

In [113]:
api_key = os.environ.get('binance_key')
api_secret = os.environ.get('binance_secret')
client = Client(api_key, api_secret)

# Grabbing Data

In [114]:
def highest_volume_coins(N, base='BTC'):
    '''
    returns N tickers representing the highest volume coins
    of those with the base str in the ticker
    '''
    info = client.get_all_tickers()
    all_symbols = [d['symbol'] for d in info]
    priced_in_btc = [s for s in all_symbols if 'BTC' in s]

    tickers = client.get_orderbook_tickers()
    tickers = pd.DataFrame.from_dict(tickers)
    col_types = {
        'symbol': 'str',
        'bidPrice': 'float',
        'bidQty': 'float',
        'askPrice': 'float',
        'askQty': 'float',
    }
    tickers = tickers.astype(col_types)
    tickers['totalQty'] = tickers['bidQty'] + tickers['askQty']

    high_volume_coins = tickers[tickers['symbol'].isin(priced_in_btc)].nlargest(N, 'totalQty')
    symbols = high_volume_coins['symbol'].values

    return symbols




In [115]:
#symbols = highest_volume_coins(60)
#symbols

In [116]:
#d = datetime.datetime(2021, 4, 1)
#d.strftime("%d %b, %Y")
#klines = client.get_historical_klines('REEFBTC', Client.KLINE_INTERVAL_1DAY, str(d.timestamp()))

In [117]:
def price_histories(symbols, interval_size, start_datetime, end_datetime=None):
    '''
    gets the Open-High-Low-Close-Vol price data for the:
        tickers in the array symbols
        interval size \in {1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1mo}
        datetime objects respresenting the beginning and end of the time window
        (None for end_datetime gives data from start to the present)

    returns a dict, indexed by symbol
    the values of the dict are data tables with OHLCV entries for each point in time
    time increases as row index increases
    '''

    accepted_intervals = ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h', '12h', '1d', '3d', '1w', '1mo']
    if not interval_size in accepted_intervals:
        raise ValueError("Given kline interval size, {}, not supported".format(interval_size))
    
    interval_dict = {
        '1m': Client.KLINE_INTERVAL_1MINUTE,
        '3m': Client.KLINE_INTERVAL_3MINUTE,
        '5m': Client.KLINE_INTERVAL_5MINUTE,
        '15m': Client.KLINE_INTERVAL_15MINUTE,
        '30m': Client.KLINE_INTERVAL_30MINUTE,
        '1h': Client.KLINE_INTERVAL_1HOUR,
        '2h': Client.KLINE_INTERVAL_2HOUR,
        '4h': Client.KLINE_INTERVAL_4HOUR,
        '8h': Client.KLINE_INTERVAL_8HOUR,
        '12h': Client.KLINE_INTERVAL_12HOUR,
        '1d': Client.KLINE_INTERVAL_1DAY,
        '3d': Client.KLINE_INTERVAL_3DAY,
        '1w': Client.KLINE_INTERVAL_1WEEK,
        '1mo': Client.KLINE_INTERVAL_1MONTH,
    }
    
    
    price_histories = {}

    for s in symbols:
        start = str(start_datetime.timestamp())
        end = None
        if end_datetime:
            end = str(end_datetime.timestamp())
        klines = client.get_historical_klines(s, interval_dict[interval_size], start, end)
        klines = [(lambda l: [float(x) for x in l])(candle) for candle in klines]
        history = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades', 'taker_base_asset_vol', 'taker_quote_asset_vol', 'ignore'])
        price_histories[s] = history

    
    for symbol, df in price_histories.items():
        df['symbol'] = [symbol for i in range(len(df.index))]
        df['time (in units of {})'.format(interval_size)] = [i for i in range(len(df.index))]

    return price_histories


In [118]:
symbols = highest_volume_coins(60)
prices = price_histories(symbols, '1h', datetime.datetime(2021, 8, 1))


In [119]:
#plt.plot(prices[symbols[0]]['close'].values)

# PCA

In [120]:
def price_matrix_from_dict(price_histories, time_units=None, column='close'):
    '''
    takes a dict, price_histories, 
        where the key is a symbol, and the value is a time-indexed dataframe
    as well as an integer number of time units to include
        where the subset is taken from the end of the interval
        time_units=None will use all the data
    also takes an optional column header to pull data from
        alternatives include 'open', 'high' and 'low'

    returns an N by M matrix of coin prices, where the 1st coord is time, and 
        the second coord is the coin
    '''

    N = time_units
    if time_units is None:
        N = max([len(price_histories[s][column].values) for s in price_histories.keys()])
    M = len(price_histories.keys())

    price_matrix = np.ones([M, N])

    i = 0
    for s in price_histories.keys():
        history = price_histories[s]
        col = history[column].values
        if len(col) < N:
            print("Insufficient data found for symbol {}".format(s))
            continue
        price_matrix[i] = col
        i += 1

    price_matrix = np.transpose(price_matrix)

    shape = price_matrix.shape
    print("price matrix computed with {} points in time, {} coins".format(shape[0], shape[1]))

    return price_matrix

In [121]:
price_matrix = price_matrix_from_dict(prices)

price matrix computed with 767 points in time, 60 coins


In [122]:
def returns_from_prices(price_matrix):
    '''
    takes a matrix of prices where the first dim is time, 
        and the second dim is which coin
        (reminder, time moves forward with index in this context)
    returns a matrix of returns
        NB: if the input is N by M, the output will be N-1 by M
    '''

    N, M = price_matrix.shape
    return_matrix = np.ones([N-1, M])
    
    for i in range(N-1):
        for j in range(M):
            new_price = price_matrix[i+1][j]
            old_price = price_matrix[i][j]
            return_matrix[i][j] = (new_price - old_price) / old_price

    return return_matrix



In [123]:
return_matrix = returns_from_prices(price_matrix)

Standardizing returns

In [124]:
scaler = StandardScaler()
scaler.fit(return_matrix)

scaled_returns = scaler.transform(return_matrix)

Executing PCA

In [125]:
X = 10
pca = PCA(n_components=X)

pca.fit(scaled_returns)

PCA(n_components=10)

In [126]:
#transformed_returns = pca.transform(scaled_returns)
p_components = pca.components_

In [127]:
p_components.shape

(10, 60)

Math Break:

The return matrix is N (time units) by M (coins), the eigenportfolio matrix is X (components) by M (coins)

We want a matrix of the eigenportfolio returns, which should be X by N or N by X

X by M @ M by N = X by N

eigenportfolio returns (eport, time) = eigenportfolio matrix @ returns^T

We'll use the returns of the eigenportfolios in a risk factor model for individual coin returns later.


In [128]:
eigen_port_returns = p_components @ np.transpose(scaled_returns)

The matrix of eigenportfolio returns is X (portfolios) by N (time units)

We can multiply this by the N (time units) by M (coins) return matrix to obtain

an X by M matrix, the entries of which are the covariance of a given coin's returns with the returns of a given eigenportfolio.

We can then multiply the N by X (after a transposition) matrix of eigenportfolio returns by this X by M covariance matrix to yield the returns of each coin that we would predict from its projection onto the eigenbasis (an N by M matrix).

In order for the units to be right, we have to divide each row of the covariance matrix by the variance of the corresponding eigenportfolio.


In [129]:
coin_eigenport_covar_matrix = eigen_port_returns @ scaled_returns

In [130]:
for i in range(X):
    eigenport_variance = np.var(eigen_port_returns[i])
    coin_eigenport_covar_matrix[i] = coin_eigenport_covar_matrix[i] / eigenport_variance


In [131]:
expected_returns = np.transpose(eigen_port_returns) @ coin_eigenport_covar_matrix

In [132]:
expected_returns.shape

(766, 60)

The cumulative residuals from the PCA based estimation can then be modeled as a mean reverting process.

Specifically, we model the cumulative residuals as an Ornstein-Uhlenbeck process. To do this we fit an AR(1) autoregressive model to the cumulative residuals, and use the parameters from that model to calculate the parameters of the OU process. The S-score which will tell us how over or underpriced a coin is can then be directly calculated from these parameters.

https://stats.stackexchange.com/questions/210437/simple-question-about-ornstein-uhlenbeck-process/210443

In [133]:
residuals = scaled_returns - expected_returns
cumulative_residuals = np.cumsum(residuals, axis=0)

In [134]:
cumulative_residuals.shape

(766, 60)

In [135]:
def s_scores(cumulative_residuals, alpha=0.05):
    M = cumulative_residuals.shape[1]
    s_scores = np.ones(M)
    for i in range(M):
        if adfuller(cumulative_residuals[:, i])[1] < alpha:
            ar_model_results = AutoReg(cumulative_residuals[:, i], lags=1).fit()
            a_hat = ar_model_results.params[0]
            b_hat = ar_model_results.params[1]
            ar_residuals = ar_model_results.resid
            m_OU = a_hat / (1 - b_hat)
            sigma_OU = np.sqrt(np.var(ar_residuals) / (1 - (b_hat**2)))
            S = -1 * m_OU / sigma_OU
            s_scores[i] = S
        else:
            s_scores[i] = 0

    return s_scores

In [136]:
s_scores = s_scores(cumulative_residuals)
s_scores

/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
 

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.36479952, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.50651763, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.53566114, 0.        , 0.4245855 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [137]:
#s_score_chart = pd.DataFrame(data=np.transpose([symbols, s_scores]), columns=['symbol', 's_score'])

Summarizing our work

In [146]:
def PCA_risk_model(return_matrix, n_components, ad_fuller_alpha=0.05):
    '''
    takes a matrix of returns where the first dim is time and
        the second is which coin
    returns a vector of s-scores, one for each coin
        the scores are the result of decomposing the price movements of a given coin
        into the movements of n_components eigenportfolios found via PCA.
        the residuals from the PCA model are modeled as an Ornstein Uhlenbeck process
        and the final s-score can be interpreted as a z-score for how high or low
        the current cumulative returns are compared to what the model predicts
    '''

    scaler = StandardScaler()
    scaler.fit(return_matrix)

    scaled_returns = scaler.transform(return_matrix)

    pca = PCA(n_components=n_components)

    pca.fit(scaled_returns)

    p_components = pca.components_

    eigen_port_returns = p_components @ np.transpose(scaled_returns)

    coin_eigenport_covar_matrix = eigen_port_returns @ scaled_returns

    for i in range(X):
        eigenport_variance = np.var(eigen_port_returns[i])
        coin_eigenport_covar_matrix[i] = coin_eigenport_covar_matrix[i] / eigenport_variance

    expected_returns = np.transpose(eigen_port_returns) @ coin_eigenport_covar_matrix

    residuals = scaled_returns - expected_returns
    cumulative_residuals = np.cumsum(residuals, axis=0)    

    M = cumulative_residuals.shape[1]
    s_scores = np.ones(M)
    for i in range(M):
        if adfuller(cumulative_residuals[:, i])[1] < ad_fuller_alpha:
            ar_model_results = AutoReg(cumulative_residuals[:, i], lags=1).fit()
            a_hat = ar_model_results.params[0]
            b_hat = ar_model_results.params[1]
            ar_residuals = ar_model_results.resid
            m_OU = a_hat / (1 - b_hat)
            sigma_OU = np.sqrt(np.var(ar_residuals) / (1 - (b_hat**2)))
            S = -1 * m_OU / sigma_OU
            s_scores[i] = S
        else:
            s_scores[i] = 0

    return s_scores

In [148]:
s_scores = PCA_risk_model(return_matrix, 10, ad_fuller_alpha=0.05)
#print(s_scores)

SyntaxError: invalid syntax (880212282.py, line 2)

In [140]:
#s_score_chart = pd.DataFrame(data=np.transpose([symbols, s_scores]), columns=["Symbol", "S_Score"])
#s_score_chart.head()

Recap of workflow so far:

In [141]:
#symbols = highest_volume_coins(60)
#prices = price_histories(symbols, '1h', datetime.datetime(2021, 8, 1))
#price_matrix = price_matrix(prices)
#return_matrix = returns_from_prices(price_matrix)
#s_scores = PCA_risk_model(return_matrix, 10, ad_fuller_alpha=0.05)

Next:
- determine how best to trade off of s-scores
- backtest
- ^buy if s < a and sell if s > b, exit position when |s| < c, backtest to find reasonable values of a, b and c
- also need to backtest to see what time windows are profitable

# Backtesting

In [197]:
bt_start = datetime.datetime(2020, 1, 1)
bt_end = datetime.datetime(2021, 1, 1)
symbols = highest_volume_coins(100)
prices = price_histories(symbols, '1d', bt_start, bt_end)
price_matrix = price_matrix_from_dict(prices)
backtesting_data = price_matrix

Insufficient data found for symbol CKBBTC
Insufficient data found for symbol REEFBTC
Insufficient data found for symbol STMXBTC
Insufficient data found for symbol RSRBTC
Insufficient data found for symbol AKROBTC
Insufficient data found for symbol TCTBTC
Insufficient data found for symbol DGBBTC
Insufficient data found for symbol MDTBTC
Insufficient data found for symbol LINABTC
Insufficient data found for symbol BTCDOWNUSDT
Insufficient data found for symbol IDEXBTC
Insufficient data found for symbol STPTBTC
Insufficient data found for symbol JSTBTC
Insufficient data found for symbol FORBTC
Insufficient data found for symbol PONDBTC
Insufficient data found for symbol ROSEBTC
Insufficient data found for symbol AGIXBTC
Insufficient data found for symbol UTKBTC
Insufficient data found for symbol IRISBTC
Insufficient data found for symbol COTIBTC
Insufficient data found for symbol SKLBTC
Insufficient data found for symbol CHRBTC
Insufficient data found for symbol TLMBTC
Insufficient data 

In [198]:
X = 10 # n_components for PCA
s_sell = 2.5 # will sell if s > s_sell
s_buy = -2.5 # will buy if s < s_buy
s_close_posn = 0.5 # will unwind posn if |s| < s_close_posn
enable_shorting = False # if false, will only act on buy signals
inference_window_size = 30 # testing a 30 day window over the last year
bet_size = 0.02 # bets will be scaled to this portion of the portfolio size

In [199]:
t = 0
t_max = backtesting_data.shape[0] - inference_window_size
account_balance = 10000
trades = [] # trade has {symbol: , direction: , size: , entry_time: }
log = ""


In [200]:
while t < t_max:

    log += 't = {}\n'.format(t)
    log += 'current open positions: ' + str(trades) + '\n'

    # new day
    # get data slice
    # call pca and get s scores
    # iterate over s scores and generate trades
    # simulate trades in portfolio (including flash forward)

    data_slice = backtesting_data[t: t+inference_window_size]
    return_slice = returns_from_prices(data_slice)
    s_scores = PCA_risk_model(return_slice, 10)
    s_score_dict = {}
    for i, symbol in enumerate(symbols):
        s_score_dict[symbol] = s_scores[i]

    
    # updating trade values with return data
    # unwinding trades

    new_trades = []
    for trade in trades:
        trade['value'] *= (1 + return_slice[-1, list(symbols).index(trade['symbol'])])
        if s_score_dict[trade['symbol']]**2 < s_close_posn**2:
            account_balance += np.abs(trade['value'])
        else:
            new_trades.append(trade)
    trades = new_trades
    
    
    # making new trades based on signals

    for symbol, score in s_score_dict.items():
        bet = account_balance * bet_size
        if account_balance > 0:
            if score < s_buy:
                trades.append({'symbol': symbol, 'direction': 'Buy', 'value': bet, 'entry_time': t})
                account_balance -= bet
            if (score > s_sell) and enable_shorting: 
                trades.append({'symbol': symbol, 'direction': 'Sell', 'value': bet, 'entry_time': t})
                account_balance += bet
        

    t = t + 1

/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
 

In [201]:
print(log)

t = 0
current open positions: []
t = 1
current open positions: []
t = 2
current open positions: []
t = 3
current open positions: []
t = 4
current open positions: []
t = 5
current open positions: [{'symbol': 'ZILBTC', 'direction': 'Buy', 'value': 200.0, 'entry_time': 4}, {'symbol': 'ANKRBTC', 'direction': 'Buy', 'value': 196.0, 'entry_time': 4}]
t = 6
current open positions: [{'symbol': 'ANKRBTC', 'direction': 'Buy', 'value': 216.1025641025641, 'entry_time': 4}, {'symbol': 'XEMBTC', 'direction': 'Buy', 'value': 196.22501510574017, 'entry_time': 5}]
t = 7
current open positions: [{'symbol': 'XEMBTC', 'direction': 'Buy', 'value': 191.12089621570647, 'entry_time': 5}]
t = 8
current open positions: []
t = 9
current open positions: []
t = 10
current open positions: []
t = 11
current open positions: []
t = 12
current open positions: []
t = 13
current open positions: []
t = 14
current open positions: []
t = 15
current open positions: []
t = 16
current open positions: []
t = 17
current open pos

In [202]:
account_balance

10023.44209113742